In [2]:
import re
import csv
import os
import uuid
import json
import nltk
import spacy
import requests
import googlemaps 
import pandas as pd
from bs4 import BeautifulSoup
from art import *
import zipfile

import secret # API keys

In [2]:
from google.cloud import storage
from transformers import pipeline
from sentence_transformers import SentenceTransformer 
from transformers import AutoTokenizer, AutoModelForTokenClassification

/Users/zacharyg/miniconda3/envs/python_se/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import io
import joblib
import openai
import torch
import pickle
from bertopic import BERTopic

In [4]:
from tqdm import tqdm
from bson import ObjectId
from pymongo import MongoClient

### Census Data Class

In [5]:
"""
This is a class that takes in two csv's. One csv for census data and one for census data overall by neighborhoods.
As of this creation of this class, we are using the Boston Census 2020 data. Please make sure that future 
versions follow the same path.
"""
class census_data():
    def __init__(self, db, fetch_arr):
        self.db = db
        self.fetch_arr = fetch_arr
        self.load_census_neigh_data()
        self.load_old_census_data()
        self.load_census_data()
        
    def load_old_census_data(self):
        try:
            if (self.db == None):
                raise Exception("No database was given!")
                
            if (self.fetch_arr[2]):
                bucket = self.db.bucket("ml_naacp_model_data")
                blob = bucket.blob("Topic_Modeling_Pipeline_Data/census.json")
                blob.download_to_filename("./geodata_prod/census.json")

            old_census_tracts_db = json.load(open("./geodata_prod/census.json"))

            self.old_census_tracts = old_census_tracts_db
        except Exception as err:
            print(f"Error loading Census Data class!\nError: {err}")
            raise Exception("Fatal Error in Class Construction!")
        return
    
    def load_census_data(self, fetch=True):
        try:
            if (self.db == None):
                raise Exception("No database was given!")

            if (self.fetch_arr[1]):
                bucket = self.db.bucket("ml_naacp_model_data")
                blob = bucket.blob("Topic_Modeling_Pipeline_Data/census_2020.csv")
                blob.download_to_filename("./geodata_prod/census_2020.csv")

            census_tracts_df = pd.read_csv("./geodata_prod/census_2020.csv")
            
            self.census_tracts = self.process_census_data(census_tracts_df)
        except Exception as err:
            print(f"Error loading Census Data class!\nError: {err}")
            raise Exception("Fatal Error in Class Construction!")
        return
        
    def load_census_neigh_data(self, fetch=True):
        try:
            if (self.db == None):
                raise Exception("No database was given!")
                
            if (self.fetch_arr[0]):
                bucket = self.db.bucket("ml_naacp_model_data")
                blob = bucket.blob("Topic_Modeling_Pipeline_Data/census_2020_neigh.csv")
                blob.download_to_filename("./geodata_prod/census_2020_neigh.csv")

            census_neigh_data_df = pd.read_csv("./geodata_prod/census_2020_neigh.csv")

            self.census_neighbourhoods = self.process_census_neigh_data(census_neigh_data_df)
        except Exception as err:
            print(f"Error loading Census Data class!\nError: {err}")
            raise Exception("Fatal Error in Class Construction!")
        return

    def process_census_data(self, df):
        demographics = df.iloc[:,11:20]
        geoid_tract = df['GEOCODE']
        tract = df['TRACT']

        concat_pd = pd.concat([tract, geoid_tract, demographics], axis=1)
        concat_pd.drop(concat_pd.index[0], inplace=True)
        concat_pd = concat_pd.rename(
            columns={
                'TRACT': 'tract', 
                'GEOCODE': 'geoid_tract',
                'P0020001': 'total'        
            }
        )
        return concat_pd

    # A function for the future
    def process_census_neigh_data(self, df):
        df = df.iloc[:,:7]
        df = df.rename(
            columns={
                'tract20_nbhd': 'Neighborhood',       
            }
        )
        df.drop(df.index[0], inplace=True)
        return df

### Neighbourhood Mapping Class

In [6]:
class neighborhood_mapping():
    def __init__(self, db, fetch_arr):
        self.db = db
        self.fetch_arr = fetch_arr
        self.load_mappings()
    
    def load_mappings(self):
        try:
            if (self.db == None):
                raise Exception("No database was given!")
                
            if (self.fetch_arr[0]):
                bucket = self.db.bucket("ml_naacp_model_data")
                blob = bucket.blob("Entity_Recognition_Pipeline_Data/tracts-neighbors.json")
                blob.download_to_filename("./geodata_prod/tracts-neighbors.json")

            if (self.fetch_arr[1]):
                bucket = self.db.bucket("ml_naacp_model_data")
                blob = bucket.blob("Entity_Recognition_Pipeline_Data/blocks-neighbors.json")
                blob.download_to_filename("./geodata_prod/blocks-neighbors.json")

            block_map_db = json.load(open("./geo-data/blocks-neighbors.json"))
            tract_map_db = json.load(open("./geodata_prod/tracts-neighbors.json"))
            
            self.tract_mapping = tract_map_db
            self.block_mapping = block_map_db
        except Exception as err:
            print(f"Error loading neighborhood mapping class!\nError: {err}")
            raise Exception("Fatal Error in Class Construction!")
        return

    def tract_to_neighborhood(self, tract):
        # given a census tract return the boston neighborhood it is in 
        return self.tract_mapping[tract]

    def block_to_neighborhood(self, block):
        # given a census block return the boston neighborhood it is in 
        return self.block_mapping(block)

### Geography Class

In [7]:
class geography():
    def __init__(self, db, fetch_arr):
        self.db = db
        self.fetch_arr = fetch_arr
        self.load_mass_town_entities()
        self.load_state_entities()
        self.load_saved_geocodes()
        self.load_org_entities()

    def load_saved_geocodes(self):
        try:
            if (self.db == None):
                raise Exception("No database was given!")
                
            if (self.fetch_arr[0]):
                bucket = self.db.bucket("ml_naacp_model_data")
                blob = bucket.blob("Entity_Recognition_Pipeline_Data/saved-geocodes.json")
                blob.download_to_filename("./geodata_prod/saved-geocodes.json")
                
            saved_geocodes_db = json.load(open("./geodata_prod/saved-geocodes.json"))
            
            self.saved_geocodes = saved_geocodes_db
        except Exception as err:
            print(f"Error loading geography class!\nError: {err}")
            raise Exception("Fatal Error in Class Construction!")
        return
        
    def load_state_entities(self, fetch=True):
        try:
            if (self.db == None):
                raise Exception("No database was given!")

            if (self.fetch_arr[1]):
                bucket = self.db.bucket("ml_naacp_model_data")
                blob = bucket.blob("Entity_Recognition_Pipeline_Data/states.csv")
                blob.download_to_filename("./geodata_prod/states.csv")
            
            states = pd.read_csv("./geodata_prod/states.csv")
            
            states_set = set()
            for idx in range(len(states)):
                tup = (states['state'][idx], 'LOC')
                states_set.add(tup)
            self.state_entities = states_set
            
        except Exception as err:
            print(f"Error loading geography class!\nError: {err}")
            raise Exception("Fatal Error in Class Construction!")
        return
    
    def load_mass_town_entities(self, fetch=True):
        try:
            if (self.db == None):
                raise Exception("No database was given!")
                
            if (self.fetch_arr[2]):
                bucket = self.db.bucket("ml_naacp_model_data")
                blob = bucket.blob("Entity_Recognition_Pipeline_Data/mass-towns.csv")
                blob.download_to_filename("./geodata_prod/mass-towns.csv")
                
            towns = pd.read_csv("./geodata_prod/mass-towns.csv")

            towns_set = set()
            for idx in range(len(towns)):
                tup = (towns['town'][idx], 'LOC')
                towns_set.add(tup)
            self.mass_town_entities = towns_set
        except Exception as err:
            print(f"Error loading geography class!\nError: {err}")
            raise Exception("Fatal Error in Class Construction!")
        return
    
    def load_org_entities(self):
        self.org_entities = (('GBH News', 'ORG'), ('Boston Public Radio', 'ORG'), 
                             ('Supreme Court', 'ORG'), ('New York Times', 'ORG'), 
                             ('Washington Post', 'ORG'), ('CNN', 'ORG'), 
                             ('NPR', 'ORG'), ('Associated', 'ORG'), 
                             ('Press', 'ORG'), ('Senate', 'ORG'), 
                             ('Associated Press', 'ORG'), ('AP', 'ORG'), 
                             ('ABC News', 'ORG'),('CSS', 'ORG'), 
                             ('Philadelphia Inquirer', 'ORG'), ('House', 'ORG'),
                             ('Congress', 'ORG'), ('Worcester', 'ORG'),
                             ('FBI', 'ORG'), ('Homeland Security Department', 'ORG'),
                             ('CDC', 'ORG'),('Fox News', 'ORG'),('The Washington Post', 'ORG'),
                             ('States', 'LOC'), ('S.', 'LOC'), ('Massachusetts', 'ORG'),
                             ('White House', 'ORG'), ('High School', 'ORG'),
                             ('MIT', 'ORG'), ('Harvard University', 'ORG'),
                             ('White House', 'LOC'),('Greater Boston', 'LOC'),
                             ('New England', 'LOC'))

In [8]:
def load_bert_NER():
    # loading bert NER model 
    tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
    model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")
    nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="max")

    return nlp

In [9]:
def load_bert_TOPIC():
    #loading bert Topic model
    topic_model = BERTopic.load(
        "./llm_models/bglobe_519_body_230_cereal", 
        embedding_model=SentenceTransformer("all-MiniLM-L6-v2")
    )
    return topic_model

In [10]:
def get_locations_bert(article_text, nlp):
    """
    get location names from article using NER - bert model 
    https://huggingface.co/dslim/bert-base-NER
    input: article_text as a string, aggregate of h1, h2, lede, and body
    returns: locations - set of tuples of (NAME, 'LOC') and organizations - set of tuples (NAME, 'ORG) mentioned in the article
    """
    
    ner_results = nlp(article_text)
    locations = set([(X['word'],X['entity_group']) for X in ner_results if X['entity_group'] == 'LOC'])
    orgs = set([(X['word'], X['entity_group']) for X in ner_results if X['entity_group'] == 'ORG'])

    return locations, orgs

def clean_entity_results(extracted_loc, extracted_orgs, drop_geos):
    # cleaning extracted entities from bert 
    # removing state names, and mass town names since the demographics data is too broad
    # return cleaned set of entities
    entity_result = extracted_loc | extracted_orgs

    for tup in extracted_loc | extracted_orgs:
        if len(tup[0]) <= 1:
            entity_result.remove(tup)
        elif tup in drop_geos.state_entities:
            entity_result.remove(tup)
        elif tup in drop_geos.mass_town_entities:
            entity_result.remove(tup)
        elif tup in drop_geos.org_entities:
            entity_result.remove(tup)
    return entity_result

def remove_existing_geocodes(entity_result, saved_geocodes):
    # check if any locations or organizations were recognized
    # check if the geocodes already exist in dictionary
    existing_loc_geocode = {}
    new_loc_geocode = set()
    for ent in entity_result:
        try:
            existing_loc_geocode[ent[0]] = saved_geocodes[ent[0]]
        except KeyError:
            new_loc_geocode.add(ent)
    return existing_loc_geocode, new_loc_geocode

def get_snippet(sentences, num_sent, lede=True, remaining_text=False):
    """
    get the snippet of text from the article_text, replace single quotes
    input: article text, and num_sent - number of sentences to return, default lede is true will return first x sentences
           reamaining_text then must be False 
    returns: first x (num_sent) sentences
    """
    #clean_text = clean_article_text(text)
    #clean_text = ". ".join(clean_text.split(".")) # adding a space after period so nltk can do a better job recognizing sentences
    #lede = nltk.sent_tokenize(clean_text)[:num_sent] # returns a list
    
    if lede: # get the first num_sent 
        lede_text = sentences[:num_sent]
        result_text = " ".join(lede_text)
    elif remaining_text: # get rest of article num_sent * 2 until the end
        result_text = sentences[num_sent*2:]
        result_text = " ".join(result_text)
    else: # get sentences num_sent to num_sent * 2
       result_text = sentences[num_sent:num_sent*2]
       result_text = " ".join(result_text) 
    
    singleq = result_text.replace('’', "'")

    return singleq

def get_sentences(text):
    # return article text as a list of its sentences 

    clean_text = clean_article_text(text)
    clean_text = ". ".join(clean_text.split(".")) # adding a space after period so nltk can do a better job recognizing sentences
    sentences = nltk.sent_tokenize(clean_text)

    return sentences

def clean_article_text(text):
    # get text, removing html tags
    soup = BeautifulSoup(text, "html.parser")
    clean_text = soup.get_text()
    return clean_text

In [11]:
def get_location_geocode(API_KEY, locations):
    """
    getting coordinates from location names in articles 
    input: google maps platform API KEY, locations article 
    return: dictionary of location names (key) with coordinates (value as a dictionary with lat and lon as keys)
    """
    gmaps = googlemaps.Client(key=API_KEY)
    results = {}

    # getting coordinates
    for place in locations:
        # we can constrain google geocode api search to massachusetts or us - census geocoder will not work for places outside of U.S 
        #geocode_result = gmaps.geocode(place[0] + ", Suffok County, MA, USA") # place is a tuple, where first value is the location name 
        geocode_result = gmaps.geocode(place[0] + ", Suffolk County",  components={"administrative_area_level": "MA", 
                                                                                   "country": "US"})
        #print(geocode_result)
        #print()
        temp = {}
        try:
            geocode_components = geocode_result[0]['address_components']
            for i, addr_comp in enumerate(geocode_components):
                if 'administrative_area_level_2' in addr_comp['types']:
                    if "Suffolk County" == addr_comp['short_name'] and i != 0:
                        temp['lat'] = geocode_result[0]['geometry']['location']['lat']
                        temp['lon'] = geocode_result[0]['geometry']['location']['lng']
                        results[place[0]] = temp
                        
        except IndexError: # unable to get coordinates for location
            print("Unable to locate " + place[0])

    return results 

In [12]:
def get_census_geos(geocode_results):
    """
    get census geographies - tract, block group, block by coordinates
    input: google maps geocode_results as a dictionary
    return: block, block_group, tract, county for each location
    """
    census_geos = {}
    for place in geocode_results:
        # building the geocoding url
        base_url = f'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?'
        survey_ver = f'&benchmark=4&vintage=4&layers=2020 Census Blocks&format=json'
        lon = geocode_results[place]['lon']
        lat = geocode_results[place]['lat']
        census_geo_url = f'{base_url}x={lon}&y={lat}{survey_ver}'

        # getting the census geographies 
        response = requests.get(census_geo_url)
        response_json = response.json()

        try:
            block = response_json['result']['geographies']['2020 Census Blocks'][0]['BLOCK']
            block_group = response_json['result']['geographies']['2020 Census Blocks'][0]['BLKGRP']
            tract = response_json['result']['geographies']['2020 Census Blocks'][0]['TRACT']
            county = response_json['result']['geographies']['2020 Census Blocks'][0]['COUNTY']
            census_geos[place] = {'block': block,
                                  'blkgrp': block_group,
                                  'tract': tract,
                                  'county': county}
        except IndexError:
            print("Unable to retrieve census geography for: " + place)
        except KeyError:
            print("Location is outside of the United States: " + place)
    return census_geos

In [13]:
"""
# CENSUS DATA API 
# https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_api_handbook_2020_ch02.pdf
# any user can query small quantities of data with minimal restrictions - up to 50 variables in a single query, up to 500 queries per IP address per day 
# more than 500 queries per IP address per day requires you to register for API key - www.census.gov/developers
# https://www.census.gov/data/developers/data-sets/decennial-census.html 
"""
def get_census_demographics(year, dsource, dname, tract, county, state):
    # input: census year, data source, survey name, tract, county, state
    # return: demographic data for tract mentioned
    
    # census variables: https://api.census.gov/data/2020/dec/pl/variables.html 
    cols = 'NAME,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,P2_009N,P2_010N'
    base_url = f"https://api.census.gov/data/{year}/{dsource}/{dname}"

    # to get tract demographics 
    census_url = f"{base_url}?get={cols}&for=tract:{tract}&in=county:{county}&in=state:{state}"

    # to get block demographics 
    # census_url = f"{base_url}?get={cols}&for=block:{block}&in=tract:{tract}&in=county:{county}&in=state:{state}"

    census_response = requests.get(census_url)
    census_response_json = census_response.json()

    return census_response_json

def run_entity_recognition(text, nlp, drop_geos, saved_geocodes):
    # running entity recogntion on text
    # parse existing geocoded entities and new geocoded entities
    try:
        extracted_loc, extracted_orgs = get_locations_bert(text, nlp)
        ent_result = clean_entity_results(extracted_loc, extracted_orgs, drop_geos)
        existing_loc_geocode, new_loc_geocode = remove_existing_geocodes(ent_result, saved_geocodes)
    except TypeError as e:
        print(f"No entities: {e}")
        existing_loc_geocode = {}
        new_loc_geocode = set()

    return existing_loc_geocode, new_loc_geocode

def run_location_geocode(API_KEY, new_loc_geocode):
    # get geocodes for NEW locations and saving them to json
    # returns new location geocodes as dictionary 
    location_geocode = {}
    if new_loc_geocode:
        location_geocode = get_location_geocode(API_KEY, new_loc_geocode)
    return location_geocode

def check_snippets(API_KEY, new_entities, existing_entities):
    location_geocode = run_location_geocode(API_KEY, new_entities)
    existing_loc_geocode = existing_entities
    combined_geocodes = location_geocode | existing_loc_geocode # if this is empty, then try the next snippet of text 
    return (not combined_geocodes), location_geocode, existing_loc_geocode

In [14]:
def run_pipeline(year, dsource, dname, state, existing_loc_geocode, location_geocode, mappings):
    #location_geocode = {'Boston': {'lat': 42.3600825, 'lon': -71.0588801}, 'Massachusetts': {'lat': 42.4072107, 'lon': -71.3824374}, 'Boston city': {'lat': 42.3600825, 'lon': -71.0588801}, 'Roxbury': {'lat': 42.3125672, 'lon': -71.0898796}, 'Fitchburg': {'lat': 42.5834228, 'lon': -71.8022955}, 'Medford': {'lat': 42.4184296, 'lon': -71.1061639}}
    #location_geocode = {'Massachusetts': {'lat': 42.4072107, 'lon': -71.3824374}, 'Salem': {'lat': 42.5197473, 'lon': -70.8954626}, 'Salem City Hall': {'lat': 42.5218851, 'lon': -70.8956157}}
    #location_geocode = {'Salem': {'lat': 42.5197473, 'lon': -70.8954626}, 'Massachusetts': {'lat': 42.4072107, 'lon': -71.3824374}, 'Salem City Hall': {'lat': 42.5218851, 'lon': -70.8956157}}

    #print(location_geocode | existing_loc_geocode)
    
    census_geos = get_census_geos(location_geocode | existing_loc_geocode)

    result = []
    for place_name in census_geos:
        place_info = {}
        county = census_geos[place_name]['county']
        tract = census_geos[place_name]['tract']
        
        try:
            demographic_results = get_census_demographics(year, dsource, dname, tract, county, state)

            # build result dictionary 
            place_info[place_name] = {'county_code': county} 
            place_info[place_name] = {'county_name': demographic_results[1][0]}
            place_info[place_name]['tract'] = tract
            geoid_tract = state + county + tract # this includes the state and county and tract number
            place_info[place_name]['geoid_tract'] = geoid_tract

            if mappings.tract_mapping.get(geoid_tract): # get corresponding boston neighborhood 
                place_info[place_name]['neighborhood'] = mappings.tract_mapping[state + county + tract]
            
            place_info[place_name]['demographics'] = {
                'p2_001n': demographic_results[1][1], # total population 
                'p2_002n': demographic_results[1][2], # total hispanic or latino 
                'p2_003n': demographic_results[1][3], # total not hispanic or latino 
                'p2_004n': demographic_results[1][4], # total not hispanic or latino - pop of one race
                'p2_005n': demographic_results[1][5], # total not hispanic or latino - pop of one race - white alone 
                'p2_006n': demographic_results[1][6], # total not hispanic or latino - pop of one race - black or african american alone
                'p2_007n': demographic_results[1][7], # total not hispanic or latino - pop of one race - american indian and alaska native alone
                'p2_008n': demographic_results[1][8], # total not hispanic or latino - pop of one race - asian alone 
                'p2_009n': demographic_results[1][9], # total not hispanic or latino - pop of one race - native hawaiian and other pacific islander alone
                'p2_010n': demographic_results[1][10] # total not hispanic or latino - pop of one race - some other race alone 
            } 
            
            result.append(place_info)
        except Exception as e:
            print(e)
            print("Unable to get census demographics for: " + place_name)
    
    return result

### Bootstrap the pipeline | The functions

Next we have to bootstrap the pipeline and make sure that BERT is up and running/loaded and the variables needed is all good to go.

In [15]:
import sys
import time
import itertools
import threading

class Spinner:
    def __init__(self, message, delay=0.1):
        self.spinner = itertools.cycle(['-', '/', '|', '\\'])
        self.delay = delay
        self.message = message
        self.thread = threading.Thread(target=self.spin)
        self.stop_running = threading.Event()

    def spin(self):
        while not self.stop_running.is_set():
            sys.stdout.write(next(self.spinner))  # write the next character
            sys.stdout.flush()                    # flush stdout buffer (actual character display)
            sys.stdout.write('\b')                # erase the last written char
            time.sleep(self.delay)

    def start(self):
        self.stop_running.clear()
        sys.stdout.write(self.message + ' ')
        self.thread.start()

    def stop(self):
        self.stop_running.set()
        self.thread.join()                       # wait for spinner to stop
        sys.stdout.write('✔️ OK\n')              # write final message
        sys.stdout.flush()

    def err(self):
        self.stop_running.set()
        self.thread.join()                       # wait for spinner to stop
        sys.stdout.write('❌ Error!\n')              # write final message
        sys.stdout.flush()

In [16]:
def fetch_llm_models():
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket("naacp-models")
        blob = bucket.blob("BERTopic_Models/bglobe_519_body_230_cereal.zip")
        blob.download_to_filename("./llm_models/bglobe_519_body_230_cereal.zip")

        destination_file_name = "./llm_models/bglobe_519_body_230_cereal.zip"
        with zipfile.ZipFile(destination_file_name, 'r') as zip_ref: # Extract the ZIP
            zip_ref.extractall("./llm_models")
        os.remove(destination_file_name) # Remove the ZIP for cleanup
    except Exception as e:
        print(f"Model fetching failed! {e}")
        raise Exception("Fatal Error in Fetching LLM models. Exiting...")
    return

In [17]:
def load_heirarch_data(db, fetch=True):
    try:
        if (db == None):
            raise Exception("No database was given!")
        if (fetch):
            bucket = db.bucket("ml_naacp_model_data")
            blob = bucket.blob("Topic_Modeling_Pipeline_Data/openai_label_from_taxonomy_structured_230.json")
            blob.download_to_filename("./geodata_prod/openai_label_from_taxonomy_structured_230.json")

        heirarch_db = json.load(open("./geodata_prod/openai_label_from_taxonomy_structured_230.json"))
        
        if (heirarch_db == None):
            raise Exception(f"heirarch_db returned None!")
            
        return heirarch_db
    except Exception as err:
        print(f"Error loading hierarchal data!\nError: {err}")
        raise Exception("Fatal Error in fetching hierarchal data!")
    return

In [18]:
def bootstrap_pipeline():
    try:
        file_manifest = [
            "tracts-neighbors.json", # Neigh Mappings
            "blocks-neighbors.json", # Neigh Mappings
            "saved-geocodes.json", # Geography
            "states.csv", # Geography
            "mass-towns.csv", # Geography
            "openai_label_from_taxonomy_structured_230.json", # Topic Modeling
            "census_2020_neigh.csv", # Census
            "census_2020.csv", # Census
            "census.json" # Census
        ]
        dependency_resolver_arr = []
        
        tprint("BU Spark!", font="sub-zero")
        print("Bootstrapping pipeline... (This should only run once!)")
        print()
        spinner = Spinner("Setting up variables...")
        spinner.start()
        year='2020'
        dsource='dec' # which survey are we interested in ? decennial 
        dname='pl' # a dataset within a survey, pl - redistricting data 
        state='25' # state code 
        spinner.stop()
        print()

        spinner = Spinner("Connecting to MongoDB...")
        spinner.start()
        client = MongoClient(secret.MONGO_URI_NAACP) # A mock connection to catch early errors
        spinner.stop()
        print()

        spinner = spinner = Spinner("Connecting to Google Cloud Storage Bucket...")
        spinner.start()
        db = storage.Client()
        spinner.stop()
        print()
        
        spinner = Spinner("Detecting & Making local directories for Models...")
        spinner.start()
        spinner.stop()
        llm_model_directory_path = "./llm_models"
        if (not os.path.exists(llm_model_directory_path)):
            print(f"No {llm_model_directory_path}! Creating...")
            os.makedirs(llm_model_directory_path)
            spinner = Spinner("Fetching models...")
            spinner.start()
            fetch_llm_models()
            spinner.stop()
        else:
            print(f"Found! {llm_model_directory_path}!")
            spinner = Spinner("Validating model files...")
            spinner.start()
            if (not os.path.isfile("./llm_models/BERTopic_CPU_M1")):
                spinner.stop()
                spinner = Spinner("Model file not found! Pulling models...")
                spinner.start()
                fetch_llm_models()
                spinner.stop()
            else:
                spinner.stop()

        print("Model files successfully validated.\n")

        spinner = Spinner("Detecting & Making local directories for Geographic Data...")
        spinner.start()
        spinner.stop()
        geodata_directory_path = "./geodata_prod"
        if (not os.path.exists(geodata_directory_path)):
            print(f"No {geodata_directory_path}! Creating...")
            os.makedirs(geodata_directory_path)
            dependency_resolver_arr = len(file_manifest) * [True]
        else:
            print(f"Found! {geodata_directory_path}!")
            spinner = Spinner("Validating geodata files...")
            spinner.start()
            for file in file_manifest:
                if (not os.path.isfile(f"./geodata_prod/{file}")):
                    dependency_resolver_arr.append(True)
                else:
                    dependency_resolver_arr.append(False)
            spinner.stop()
        print("Geodata files successfully validated and updated dependecy array.\n")
        
        spinner = Spinner("Fetching OpenAI Hierarchal Mappings...")
        spinner.start()
        heir_data = load_heirarch_data(
            db,
            fetch=dependency_resolver_arr[5]
        )
        spinner.stop()
        print()
        
        spinner = Spinner("Instantiating classes...")
        spinner.start()
        mappings = neighborhood_mapping(
            db=db,
            fetch_arr=dependency_resolver_arr[:2]
        )
        drop_geos = geography(
            db=db,
            fetch_arr=dependency_resolver_arr[2:5] 
        )
        census_base = census_data(
            db=db,
            fetch_arr=dependency_resolver_arr[6:] 
        )
        saved_geocodes = drop_geos.saved_geocodes 
        spinner.stop()
        print()
        
        spinner = Spinner("Loading Models...")
        spinner.start()
        nlp_ner = load_bert_NER()
        nlp_topic = load_bert_TOPIC()
        spinner.stop()
        
        print("\nBootstrap complete!\n")
        return year, dsource, dname, state, drop_geos, mappings, census_base, heir_data, saved_geocodes, nlp_ner, nlp_topic, db
    except Exception as e:
        spinner.err()
        print(f"Bootstrap Failed!!!\nFatal Error:{e}")
        raise Exception("Fatal Error in Bootstrapping ML Pipeline. Exiting...")
        
    return

In [19]:
def validate_bootstrap(year, dsource, dname, state, drop_geos, mappings, census_base, heir_data, saved_geocodes, nlp_ner, nlp_topic, db):
    try:
        spinner = Spinner("Validating Bootstrap variables...")
        spinner.start()

        variable_manifest = {
            "year":year, 
            "dsource": dsource, 
            "dname": dname, 
            "state": state,
            "drop_geos": drop_geos,
            "mappings": mappings,
            "census_base": census_base,
            "heir_data": heir_data,
            "saved_geocodes": saved_geocodes, 
            "nlp_ner": nlp_ner, 
            "nlp_topic": nlp_topic,
            "db": db
        }
        
        for var in variable_manifest.keys():
            if (variable_manifest[var] == None):
                raise Exception(f"{var} returned None!. Exiting...")
        spinner.stop()
        print()
        print("Validation Complete! Everything seems to be in order.")
    except Exception as e:
        spinner.err()
        print(f"Bootstrap Validation Failed!!!\nFatal Error:{e}")
        raise Exception("Fatal Error in Bootstrapping Validation. Exiting...")
        

# Bootstrapping

Ideally, we should only be concerned with articles that yield a recognition of location. If not, we chuck it out!.

I think we should append it to the end of the df to make a cleaner since this needs to be passed to topic modeling.

In [20]:
#Bootstrap Run
year, dsource, dname, state, drop_geos, mappings, census_base, heir_data, saved_geocodes, nlp_ner, nlp_topic, db = bootstrap_pipeline()

 ______     __  __        ______     ______   ______     ______     __  __    
/\  == \   /\ \/\ \      /\  ___\   /\  == \ /\  __ \   /\  == \   /\ \/ /    
\ \  __<   \ \ \_\ \     \ \___  \  \ \  _-/ \ \  __ \  \ \  __<   \ \  _"-.  
 \ \_____\  \ \_____\     \/\_____\  \ \_\    \ \_\ \_\  \ \_\ \_\  \ \_\ \_\ 
  \/_____/   \/_____/      \/_____/   \/_/     \/_/\/_/   \/_/ /_/   \/_/\/_/ 
                                                                              

Bootstrapping pipeline... (This should only run once!)

Setting up variables... ✔️ OK

Connecting to MongoDB... ✔️ OK

Connecting to Google Cloud Storage Bucket... ✔️ OK

Detecting & Making local directories for Models... ✔️ OK
Found! ./llm_models!
Validating model files... ✔️ OK
Model file not found! Pulling models... ✔️ OK
Model files successfully validated.

Detecting & Making local directories for Geographic Data... ✔️ OK
Found! ./geodata_prod!
Validating geodata files... ✔️ OK
Geodata files successfully validated a

Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


✔️ OK

Bootstrap complete!



In [21]:
validate_bootstrap(year, dsource, dname, state, drop_geos, mappings, census_base, heir_data, saved_geocodes, nlp_ner, nlp_topic, db)

Validating Bootstrap variables... ✔️ OK

Validation Complete! Everything seems to be in order.


### Some Useful MongoDB functions for connections

In [22]:
def connect_MongoDB_Prod():
    try:
        client = MongoClient(secret.MONGO_URI_NAACP)
        db = client['se_naacp_db']
        print("[INFO] Connected to MongoDB Production Database!")
        return db
    except Exception as err:
        raise Exception("[Fatal Error!] Failed to Connect to MongoDB. [No retry implemented]")
    return 

def bootstrap_MongoDB_Prod(db_prod, defined_collection_names):
    """
    Adds the upload collection and other necessities that both the GraphQL and AI Pipeline share.
    Sets up the databse.
    """
    try:
        spinner = Spinner("Checking and Bootstrapping Production DB...\n")
        spinner.start()
        if (db_prod == None):
            raise Exception("No database was given!")
            
        # Here we check for the upload collection and make it if it doesn't exist
        collection_list = db_prod.list_collection_names()
        for collection in defined_collection_names:
            if collection not in collection_list:
                db_prod.create_collection(collection)
                print(f"[INFO] Collection '{collection}' created.\n")
        spinner.stop()
    except Exception as err:
        spinner.err()
        print(f"[Error!] Error in Bootstrapping MongoDB Prod DB\nError: {err}")
        raise Exception("Fatal Error in MongoDB Boostrap")
    return

In [23]:
db_prod = connect_MongoDB_Prod()

[INFO] Connected to MongoDB Production Database!


In [24]:
defined_collection_names = ["uploads", "discarded"]

bootstrap_MongoDB_Prod(db_prod, defined_collection_names)

Checking and Bootstrapping Production DB...
 ✔️ OK


# The Pipeline

### Incoming CSV

In [25]:
df = pd.read_csv("./Articles Nov 2020 - March 2023.csv", low_memory=False)
df = df.iloc[:, : 12]
df.shape

(12795, 12)

In [26]:
df

,Type,Label,Headline,Byline,Section Navigation,Section,Tagging,Title,Paths,Publish Date,Has Path?,Body
0,Article,A Celtic Playlist For Easter,A Celtic Playlist For Easter,Brian O'Donovan,NaN,Celtic,0000016a-3bcb-d661-af7b-7bff0d200001,NaN,/music/celtic/2019/04/20/a-celtic-playlist-for...,Mon Mar 29 15:22:35 EDT 2021,TRUE,The above is a continuous stream. <br/><br/>I...
1,Article,Songs Of War And Remembrance: Memorial Day,Songs Of War And Remembrance: Memorial Day,Brian O'Donovan,NaN,Celtic,0000016a-f0d5-dbfd-a56f-f4dfc34c0001,NaN,/music/celtic/2019/05/25/songs-of-war-and-reme...,Sat May 29 01:00:45 EDT 2021,TRUE,Click above for the audio of a special segment...
2,Article,Words And Music: Father's Day,Words And Music: Father's Day,Brian O'Donovan,NaN,Celtic,0000016b-539d-d757-adef-f7bd4a5f0001,NaN,/music/celtic/2019/06/13/words-and-music-fathe...,Fri Jun 18 01:00:28 EDT 2021,TRUE,"In honor of Father&#39;s Day, this segment of ..."
3,Article,Celebrating The Birthday Of Robert Burns — Jan...,Celebrating The Birthday Of Robert Burns — Jan...,Brian O'Donovan,NaN,Celtic,0000016f-c48a-d14c-a57f-e59b02310001,NaN,/music/celtic/2020/01/21/celebrating-the-birth...,Sat Jan 23 08:47:27 EST 2021,TRUE,Robert Burns is known as &quot;Scotland&#39;s ...
4,Article,Ten Celtic Love Songs For St. Valentine's Day,Ten Celtic Love Songs For St. Valentine's Day,Brian O'Donovan,NaN,Celtic,00000170-273a-d4d2-a378-677e83f60001,NaN,/celtic/ValentinesDay (Permalink),Wed Feb 10 12:09:13 EST 2021,TRUE,"Folk music generally, and Celtic music, in par..."
...,...,...,...,...,...,...,...,...,...,...,...,...
12790,Article,5 key takeaways from the Trump indictment news,5 key takeaways from the Trump indictment news,"Emily Olson, Emma Bowman",NaN,National News,00000187-3753-da17-afc7-f77b1ea40001,NaN,/national-news/2023/03/31/5-key-takeaways-from...,Fri Mar 31 06:26:00 EDT 2023,TRUE,Former president Donald Trump has been indicte...
12791,Article,Harvard professor says government should pause...,Harvard professor says government should pause...,Alexi Cohan,NaN,Science and Technology,00000187-37f6-d65f-a1f7-bffee1410001,NaN,/science-and-technology/2023/03/31/harvard-pro...,Fri Mar 31 11:06:14 EDT 2023,TRUE,Artificial intelligence has advanced rapidly i...
12792,Article,Why Trump isn't the first president to face ar...,Why Trump isn't the first president to face ar...,"Dustin Jones, Kaitlyn Radde",NaN,National News,00000187-37f7-da17-afc7-f7ffec2a0001,NaN,/national-news/2023/03/31/why-trump-isnt-the-f...,Fri Mar 31 09:05:00 EDT 2023,TRUE,Former President Donald Trump was indicted Thu...
12793,Article,These cockroaches tweaked their mating rituals...,These cockroaches tweaked their mating rituals...,Ari Daniel,NaN,News,00000187-382e-da17-afc7-f96fcf4c0001,NaN,/news/2023/03/31/these-cockroaches-tweaked-the...,Fri Mar 31 10:02:00 EDT 2023,TRUE,Human attempts to kill cockroaches with sugary...


In [27]:
df.columns

Index(['Type', 'Label', 'Headline', 'Byline', 'Section Navigation', 'Section',
       'Tagging', 'Title', 'Paths', 'Publish Date', 'Has Path?', 'Body'],
      dtype='object')

In [28]:
df = df.drop(columns=[
    'Has Path?',
    'Title',
    'Section Navigation',
])

In [29]:
list(df.columns)

['Type',
 'Label',
 'Headline',
 'Byline',
 'Section',
 'Tagging',
 'Paths',
 'Publish Date',
 'Body']

In [30]:
df

,Type,Label,Headline,Byline,Section,Tagging,Paths,Publish Date,Body
0,Article,A Celtic Playlist For Easter,A Celtic Playlist For Easter,Brian O'Donovan,Celtic,0000016a-3bcb-d661-af7b-7bff0d200001,/music/celtic/2019/04/20/a-celtic-playlist-for...,Mon Mar 29 15:22:35 EDT 2021,The above is a continuous stream. <br/><br/>I...
1,Article,Songs Of War And Remembrance: Memorial Day,Songs Of War And Remembrance: Memorial Day,Brian O'Donovan,Celtic,0000016a-f0d5-dbfd-a56f-f4dfc34c0001,/music/celtic/2019/05/25/songs-of-war-and-reme...,Sat May 29 01:00:45 EDT 2021,Click above for the audio of a special segment...
2,Article,Words And Music: Father's Day,Words And Music: Father's Day,Brian O'Donovan,Celtic,0000016b-539d-d757-adef-f7bd4a5f0001,/music/celtic/2019/06/13/words-and-music-fathe...,Fri Jun 18 01:00:28 EDT 2021,"In honor of Father&#39;s Day, this segment of ..."
3,Article,Celebrating The Birthday Of Robert Burns — Jan...,Celebrating The Birthday Of Robert Burns — Jan...,Brian O'Donovan,Celtic,0000016f-c48a-d14c-a57f-e59b02310001,/music/celtic/2020/01/21/celebrating-the-birth...,Sat Jan 23 08:47:27 EST 2021,Robert Burns is known as &quot;Scotland&#39;s ...
4,Article,Ten Celtic Love Songs For St. Valentine's Day,Ten Celtic Love Songs For St. Valentine's Day,Brian O'Donovan,Celtic,00000170-273a-d4d2-a378-677e83f60001,/celtic/ValentinesDay (Permalink),Wed Feb 10 12:09:13 EST 2021,"Folk music generally, and Celtic music, in par..."
...,...,...,...,...,...,...,...,...,...
12790,Article,5 key takeaways from the Trump indictment news,5 key takeaways from the Trump indictment news,"Emily Olson, Emma Bowman",National News,00000187-3753-da17-afc7-f77b1ea40001,/national-news/2023/03/31/5-key-takeaways-from...,Fri Mar 31 06:26:00 EDT 2023,Former president Donald Trump has been indicte...
12791,Article,Harvard professor says government should pause...,Harvard professor says government should pause...,Alexi Cohan,Science and Technology,00000187-37f6-d65f-a1f7-bffee1410001,/science-and-technology/2023/03/31/harvard-pro...,Fri Mar 31 11:06:14 EDT 2023,Artificial intelligence has advanced rapidly i...
12792,Article,Why Trump isn't the first president to face ar...,Why Trump isn't the first president to face ar...,"Dustin Jones, Kaitlyn Radde",National News,00000187-37f7-da17-afc7-f7ffec2a0001,/national-news/2023/03/31/why-trump-isnt-the-f...,Fri Mar 31 09:05:00 EDT 2023,Former President Donald Trump was indicted Thu...
12793,Article,These cockroaches tweaked their mating rituals...,These cockroaches tweaked their mating rituals...,Ari Daniel,News,00000187-382e-da17-afc7-f96fcf4c0001,/news/2023/03/31/these-cockroaches-tweaked-the...,Fri Mar 31 10:02:00 EDT 2023,Human attempts to kill cockroaches with sugary...


In [31]:
# We predetermine the userID and upload ID
# This should have already been configured per CSV upload
# Generate a unique UUID
unique_id = str(uuid.uuid4())
userID = "1"

print(unique_id)

6710adf8-ec86-49ef-85a4-75d5a1060e29


## Start of the Pipeline

First we create a user ID and upload the state...

### Single Stream Cocurrent Processing

In [34]:
def process_data(chunk, df, data_schema, data_packaging_scheme, nlp_ner):
    """
    Processes a chunk of indices from the given dataset.
    
    Parameters
    ----
    df: The pandas dataframe that entity recognition is being done on.
    chunk: The chunk of indices to proceses in the given df.

    Returns
    ---- 
    A list of dictionary items that constitute data for an article.
    """
    ignore_article_types = ["National News", "International News", "Programs", "Digital Mural", "Jazz", "Celtic"]

    discarded_articles = []
    dataset_df = data_schema
    neighborhoods = set()
    census_tracts = set()
    try: 
        # for idx in tqdm(chunk, desc='Processing Entity Recognition'):
        for idx in chunk:
            print(idx)
            # Maybe nested 'try:' are cursed
            try:
                if df['Section'][idx] not in ignore_article_types and df['Type'][idx] == 'Article':
                    headline = str(df['Label'][idx])
                    text = str(df['Body'][idx])
                    
                    sentences = get_sentences(text)
            
                    # # get lede first 5 sentences, can change the number of sentences
                    text_5 = get_snippet(sentences, 5)
                    text_10 = get_snippet(sentences, 5, False) # get sentences 5-10
                    text_remain = get_snippet(sentences, 5, False, True)
            
                    # get entities, returns existing entities that have been seen before and new entities as sets 
                    check_order = [
                        (run_entity_recognition(headline, nlp_ner, drop_geos, saved_geocodes), "headline"), 
                        (run_entity_recognition(text_5, nlp_ner, drop_geos, saved_geocodes), "first 5 sentences"), 
                        (run_entity_recognition(text_10, nlp_ner, drop_geos, saved_geocodes), "next 5 sentences"),
                        (run_entity_recognition(text_remain, nlp_ner, drop_geos, saved_geocodes), "remaining text")
                    ]
            
                    for (entities, method) in check_order:
                        check_text, location_geocode, existing_loc_geocode = check_snippets(secret.API_KEY, entities[1], entities[0])
                        if not check_text:
                            discarded_articles.append(df['Tagging'][idx])
                            break 
        
                    # No Census tracts we want is detected
                    if (len(existing_loc_geocode) == 0 and len(location_geocode) == 0):
                        discarded_articles.append(df['Tagging'][idx])
                        continue
                    
                    pipeline_output = run_pipeline(year, dsource, dname, state, existing_loc_geocode, location_geocode, mappings)
        
                    if (pipeline_output):
                        for output in pipeline_output:
                            if ('neighborhood' in output[list(output.keys())[0]] and 'tract' in output[list(output.keys())[0]]):
                                neighborhood = output[list(output.keys())[0]]['neighborhood']
                                census_tract = output[list(output.keys())[0]]['tract']
                                neighborhoods.add(neighborhood)
                                census_tracts.add(census_tract)
                            else:
                                print("Skipped an entry!")
                                continue
                    
                    # If we have valid entity recognition | We have both some neighborhoods and census tracts
                    if (len(neighborhoods) != 0 and len(census_tracts) != 0):
                        data_packaging_scheme(
                            dataset_df, # This is the data scheme we are using
                            df['Tagging'][idx],
                            list(neighborhoods),
                            df['Section'][idx],
                            list(census_tracts),
                            df['Byline'][idx],
                            df['Body'][idx],
                            df['Tagging'][idx],
                            df['Label'][idx],
                            df['Headline'][idx],
                            df['Publish Date'][idx],
                            "GBH", # I hard coded this as we have one main client
                            df['Paths'][idx],
                            # No Open AI Labels yet
                            method,
                            existing_loc_geocode | location_geocode
                        )
                    neighborhoods.clear()
                    census_tracts.clear()
                else:
                    discarded_articles.append(df['Tagging'][idx])
            except Exception as e: # Loop inbounded error
                print(f"[Error] process_data() ran into an error! Continuing... \n[Raw Error]: {e}")
        ## Convert to Pandas dataframe...
        new_df = pd.DataFrame(dataset_df)
        return new_df, discarded_articles
    except Exception as e: 
        print(f"[Fatal Error] process_data() ran into an Error! Data is not saved!\nRaw Error:{e}")

    return


In [35]:
import inspect

# To run the pipeline, two things we need to have defined is the data_schmea and data packing func
data_schema = {
        "id": [],
        "neighborhoods": [],
        "position_section": [],
        "tracts": [],
        "author": [],
        "body": [],
        "content_id": [],
        "hl1": [],
        "hl2": [],
        "pub_date": [],
        "pub_name": [],
        "link": [],
        "method": [],
        "ent_geocodes": []
    }

# Data packing scheme, function attributes must be len(data_schema) + 1
# Ideally, the developer should provide a function to figure out how to pack the data based on their needs
def package_data_to_dict(
    data_schema, 
    id,
    neighborhoods,
    position_section,
    tracts,
    author,
    body,
    content_id,
    hl1,
    hl2,
    pub_date,
    pub_name,
    link,
    method,
    ent_geocodes
):
    try:
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        args.pop(0) # we pop off data_schema
        for arg in args:
            data_schema[arg].append(values[arg])
    except KeyError as ke:
        print("Key not found in data schema!")
        print(f"Raw Error: {ke}")
        
    return data_schema

new_df, discarded_articles = process_data(list(range(1700, 1800)), df, data_schema, package_data_to_dict, nlp_ner)

1700
1701
1702
1703
1704
1705
1706
1707
1708
1709


KeyboardInterrupt: 

In [ ]:
discarded_articles

In [ ]:
new_df

# Topic Modeling

In [ ]:
unseen_articles = new_df
unseen_articles

In [ ]:
openai_labels = heir_data
unseen_articles = unseen_articles.dropna(subset=['content_id'])

topics, probs = nlp_topic.transform(unseen_articles['body']) # get bertopics for each article
unseen_articles['bertopic_topic_label'] = topics

In [ ]:
# add open ai label to bglobe dataframe in new column
unseen_label_name = [openai_labels[unseen_articles['bertopic_topic_label'][i]]['openai'] 
              if int(unseen_articles['bertopic_topic_label'][i]) != -1 else "" for i in range(len(unseen_articles))]
unseen_articles['openai_label'] = unseen_label_name

This will be the raw data that is saved in the ML data base for data analysis by the data science team. Naturally, we will strip the data that will allow the back-end to consume it.

### Some User ID & Upload ID Stuff 

In [ ]:
unseen_articles["userID"] = userID
unseen_articles["uploadID"] = unique_id

In [ ]:
unseen_articles.head(2)

### Format for GraphQL Database

In [ ]:
packaged_data_df = unseen_articles.drop(columns=[
    'method',
    'ent_geocodes',
    'bertopic_topic_label'
])

### Force a reconnection to Production Database

In [ ]:
db_prod = connect_MongoDB_Prod()

## Conversion to Articles

In [ ]:
articles_collection_name = "articles_data"

In [ ]:
packaged_data_df.set_index('id', inplace=True)

In [ ]:
article_dict = packaged_data_df.T.to_dict('dict')

In [ ]:
def string_to_list(s):
    if(s != ''):
        return [s]
    else:
        return []  # Return the string as a single-element list

In [ ]:
article_payload = []

for article_key in article_dict.keys():
    article = article_dict[article_key]
    if ('openai_label' not in article):
        article["openai_label"] = []
    else:
        article["openai_label"] = string_to_list(article["openai_label"])
    article_payload.append(article)

In [ ]:
len(article_payload)

In [ ]:
# Check for existence of collection
collection_list = db_prod.list_collection_names()

if articles_collection_name not in collection_list:
    db_prod.create_collection(articles_collection_name)
    print(f"Collection '{articles_collection_name}' created.")

In [ ]:
insertion_result = db_prod[articles_collection_name].insert_many(article_payload)
print("Articles Successfully inserted!")
# print("Documents have been successfully inserted with the following IDs:")
# print(insertion_result.inserted_ids)

## Conversion to Neighborhood 

In [ ]:
neigh_collection_name = "neighborhood_data"
neigh_collection = db_prod[neigh_collection_name]

In [ ]:
# Check for existence of collection
collection_list = db_prod.list_collection_names()

if neigh_collection_name not in collection_list:
    db_prod.create_collection(neigh_collection_name)
    print(f"Collection '{neigh_collection_name}' created.")

In [ ]:
# Assume that the neighborhood list we get is well ordered
# We also assume that the lengths between each list is the same
neighborhood_list = packaged_data_df['neighborhoods'].to_numpy()
tagging_list = packaged_data_df['content_id'].to_numpy()

for n_idx in range(len(neighborhood_list)):
    neigh_list = neighborhood_list[n_idx]
    
    for neigh in neigh_list:
        # Here we update the tags/articles by neighborhood
        neigh_collection.find_one_and_update(
            {'value': neigh},
            {'$addToSet': {'articles': tagging_list[n_idx]}},
            upsert = True # Creates a new document of it if it doesn't exist
        )       

## Conversion to Topics 

In [ ]:
topics_collection_name = "topics_data"
topic_collection = db_prod[topics_collection_name]

In [ ]:
# Check for existence of collection
collection_list = db_prod.list_collection_names()

if topics_collection_name not in collection_list:
    db_prod.create_collection(topics_collection_name)
    print(f"Collection '{topics_collection_name}' created.")

In [ ]:
# Assume that the topics list we get is well ordered
# We also assume that the lengths between each list is the same
topics_list = packaged_data_df['position_section'].to_numpy()
tagging_list = packaged_data_df['content_id'].to_numpy()

for n_idx in range(len(topics_list)):
    topic = topics_list[n_idx]
    
    # Here we update the tags/articles by Topics
    topic_collection.find_one_and_update(
        {'value': topic},
        {'$addToSet': {'articles': tagging_list[n_idx]}},
        upsert = True # Creates a new document of it if it doesn't exist
    )       

## Conversion to Tracts

In [ ]:
def addExistingTracts(tract_collection):
    tracts_list = []

    for tract_key in census_base.old_census_tracts['tracts_filter'].keys():
        tracts_list.append(census_base.old_census_tracts['tracts_filter'][tract_key])

    tract_collection.insert_many(tracts_list)

In [ ]:
tract_collection_name = "tracts_data"
tract_collection = db_prod[tract_collection_name]

In [ ]:
# Check for existence of collection
collection_list = db_prod.list_collection_names()

if tract_collection_name not in collection_list:
    db_prod.create_collection(tract_collection_name)
    print(f"Collection '{tract_collection_name}' created.")
    # We add existing tracts from the old census tracts
    addExistingTracts(tract_collection)

In [ ]:
# Assume that the topics list we get is well ordered
# We also assume that the lengths between each list is the same
tracts_lists = packaged_data_df['tracts'].to_numpy()
tagging_list = packaged_data_df['content_id'].to_numpy()

for n_idx in range(len(tracts_lists)):
    tract_list = tracts_lists[n_idx]

    for tract in tract_list:
        # Here we update the tags/articles by tracts
        if tract_collection.find_one({'tract': tract}):
            tract_collection.find_one_and_update(
                {'tract': tract},
                {'$addToSet': {'articles': tagging_list[n_idx]}},
            )
        else: # We didn't find one and we have to label it as unknown
            unknown_tract = {
                'tract': tract,
                'neighborhood': "unknown",
                'articles': []
            }
            tract_collection.insert_one(unknown_tract)

### Add Discarded Tags

In [ ]:
discarded_collection_name = "discarded"
discarded_collection = db_prod[discarded_collection_name]

for discarded_article in discarded_articles:
    if (discarded_collection.find_one({'uploadID': unique_id})):
        discarded_collection.find_one_and_update(
            {'uploadID': unique_id},
            {'$addToSet': {'content_ids': discarded_article}},
        )
    else: # We didn't find one and we have to label it as unknown
        new_discard = {
            'uploadID': unique_id,
            'content_ids': []
        }
        discarded_collection.insert_one(new_discard)

In [ ]:
db_prod = connect_MongoDB_Prod()

In [ ]:
neigh_collection_name = "articles_data"
articles_collection = db_prod[neigh_collection_name]

In [ ]:
def is_duplicate_article(tag, articles_collection):
	return articles_collection.find_one({"content_id": tag}) is not None

In [ ]:
is_duplicate_article("00000175-e21a-dd9a-a37d-eadf3d080001", articles_collection)

In [ ]:
neigh_collection_name = "discarded"
discarded_collection = db_prod[neigh_collection_name]

In [ ]:
def is_duplicate_discarded(tag, discarded_collection):
    return discarded_collection.count_documents({'content_ids': {'$in': [tag]}}) > 0

In [ ]:
is_duplicate_discarded("00000175-e21a-dd9a-a37d-eadf3d080001", discarded_collection)

In [2]:
import pandas as pd

In [3]:
test = pd.read_csv("./Articles Nov 2020 - March 2023.csv", low_memory = False)

In [4]:
test.shape[0]

12795

In [5]:
test = test[3000:4000]

In [6]:
concat_pd = pd.concat(
    [
        test['Type'],
        test['Label'],
        test['Headline'],
        test['Byline'],
        test['Section'],
        test['Tagging'],
        test['Paths'],
        test['Publish Date'],
        test['Body'],
    ],
    axis=1
)

In [7]:
concat_pd

,Type,Label,Headline,Byline,Section,Tagging,Paths,Publish Date,Body
3000,Article,Save The Bees: Massachusetts Issues New Rules ...,Save The Bees: Massachusetts Issues New Rules ...,Hannah Uebele,Local News,00000178-65d2-dbea-a578-ffda90a40001,/local-news/2021/03/25/save-the-bees-massachus...,Thu Mar 25 06:00:08 EDT 2021,"Spring has officially sprung in Massachusetts,..."
3001,Article,Greater Boston Full Show: 03/24/21,Greater Boston Full Show: 03/24/21,Greater Boston Staff,National News,00000178-6630-d89e-a7ff-ef3aaf8d0001,/national-news/2021/03/31/greater-boston-full-...,Wed Mar 31 21:23:34 EDT 2021,"<b><a href=""https://www.wgbh.org/news/local-ne..."
3002,Article,Mass. Ed. Commissioner Jeffrey Riley On Lettin...,Mass. Ed. Commissioner Jeffrey Riley On Lettin...,Greater Boston Staff,Local News,00000178-6633-dbea-a578-fffbc6c40001,/local-news/2021/03/24/mass-ed-commissioner-je...,Wed Mar 24 21:58:41 EDT 2021,Massachusetts education officials announced We...
3003,Article,Michelle Singletary On How To Maximize Your St...,Michelle Singletary On How To Maximize Your St...,Greater Boston Staff,National News,00000178-6633-dbea-a578-fffbd5650001,/national-news/2021/03/24/michelle-singletary-...,Wed Mar 24 21:59:42 EDT 2021,Washington Post personal finance columnist Mic...
3004,Article,Colorado Democrats Consider Ban On Assault Wea...,Colorado Democrats Consider Ban On Assault Wea...,Bente Birkeland,National News,00000178-6640-dbea-a578-ffc891d10001,/national-news/2021/03/25/colorado-democrats-c...,Wed Mar 24 17:41:00 EDT 2021,Colorado could be the next state to consider a...
...,...,...,...,...,...,...,...,...,...
3995,Article,United Holds Drawings For Free Flights For Tho...,United Holds Drawings For Free Flights For Tho...,Joe Hernandez,National News,00000179-a478-d1dc-a97f-bd7b3a7c0001,/national-news/2021/05/25/united-holds-drawing...,Tue May 25 12:23:00 EDT 2021,Krispy Kreme is giving away a complimentary gl...
3996,Article,"Baker Seeks To Preserve Outdoor Dining, Remote...","Baker Seeks To Preserve Outdoor Dining, Remote...",Chris Lisinski | State House News,Local News,00000179-a48c-dcfc-a1fd-b6dc8d1d0001,/local-news/2021/05/25/baker-seeks-to-preserve...,Tue May 25 13:29:29 EDT 2021,[Story Developing] Gov. Charlie Baker on Tuesd...
3997,Article,Mass. Changes Rules On Police In Schools As Bo...,Mass. Changes Rules On Police In Schools As Bo...,Meg Woolhouse,Education,00000179-a4a3-dcfc-a1fd-b6f38db00001,/education/2021/05/26/mass-changes-rules-on-po...,Wed May 26 17:14:06 EDT 2021,Boston sits at a crossroads when it comes to t...
3998,Article,Boston Public Radio Full Show: 5/25/21,Boston Public Radio Full Show: 5/25/21,Mackenzie Farkus,National News,00000179-a4c4-dcfc-a1fd-b6d407970001,/national-news/2021/05/25/boston-public-radio-...,Tue May 25 22:49:51 EDT 2021,Today on <i>Boston Public Radio</i>:<br/><br/>...


In [8]:
concat_pd.to_csv("./test_for_gcloud_presentation_1000.csv")